# BioSimSpace - quick demo

Import BioSimSpace - we import it as BSS as we are lazy ;-)

In [ ]:
import BioSimSpace as BSS

Create a molecule from a smiles string and parameterise it using the Amber GAFF forcefield

In [ ]:
mol = BSS.Parameters.gaff("Cc1ccccc1").getMolecule()

View the molecule using BioSimSpace's integration with NGLView

In [ ]:
view = BSS.Notebook.View(mol)
view.system()

Under the hood, BioSimSpace used RDKit to create the molecule from the smiles string, and then AmberTools to generate the GAFF parameters. BioSimSpace supports the use of other tools to parameterise molecules. We can get the full list of forcefields supported using the `forceFields` function. 

In [ ]:
BSS.Parameters.forceFields()

Here we will now create a molecule from a smiles string and parameterise using the OpenFF unconstrained 1.3.1 forcefield.

In [ ]:
mol = BSS.Parameters.openff_unconstrained_1_3_1("Cc1ccccc1").getMolecule()

It is the same process and same molecule, but now with different forcefield parameters.

In [ ]:
view = BSS.Notebook.View(mol)
view.system()

We can call other external tools to solvate this molecule. We do that by specifying the dimensions of the periodic box...

In [ ]:
box, angles = BSS.Box.truncatedOctahedron(35 * BSS.Units.Length.angstrom)

...and then calling `BSS.Solvent.model`, where `model` is the water model we want to use (in this case TIP3P). This solvates using the `gmx solvate` command from GROMACS.

In [ ]:
solvated = BSS.Solvent.tip3p(molecule=mol, box=box, angles=angles)

We now have the molecule in a water box.

In [ ]:
view = BSS.Notebook.View(solvated)
view.system()

Next we want to minimise the system. We do this by choosing the `Minimisation` protocol and setting the maximum number of steps.

In [ ]:
protocol = BSS.Protocol.Minimisation(steps=1000)

We now run the simulation by passing the system and protocol to `BSS.MD.run`.

In [ ]:
process = BSS.MD.run(solvated, protocol)

At the end of the simulation, we can get back the minimised system.

In [ ]:
minimised_system = process.getSystem(block=True)
view = BSS.Notebook.View(solvated)
view.system()

The simulation was performed by GROMACS. We can see this by looking at the process that was used...

In [ ]:
process

This `process` is a handle to the underlying GROMACS simulation that was run. It knows how to speak to GROMACS and how to get information from the output files it produced. For example, we could get the final potential energy.

In [ ]:
process.getPotentialEnergy()

We can also get the standard output from the job...

In [ ]:
process.stdout()

...or any of the files that this job created.

In [ ]:
import os
os.listdir(process.workDir())

The value of BioSimSpace is that we have the same interface for a number of molecular dynamics engines.

In [ ]:
BSS.MD.engines()

We can switch to use Amber for minimisation just by changing the `engine` parameter. 

In [ ]:
process = BSS.MD.run(solvated, protocol, engine="Amber")

This is a bit slower, so you can watch the standard output being generated in real time...

In [ ]:
process.stdout()

...and even view snapshots during the process.

In [ ]:
view = BSS.Notebook.View(process)
view.system()

But, at the end, we still get the minimimised structure, in the same format that we can then pass onto the next stage of the workflow.

In [ ]:
minimised_system = process.getSystem(block=True)